<h3>Welcome to week 1 </h3>

* Importing the libraries used

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

* Reading csv files to panda dataframes
* Visualizing the dataframe columns


In [2]:
failure_data= pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')


* Dropping the empty rows out of dataframe.

In [3]:
failure_data.dropna(how='all',inplace=True)
failure_data.describe()
test_data.dropna(how='all',inplace=True)

* We eliminate the object attribute for object classification as it can't be a factor to
    decide object failure since its fixed for a object type
* Filling measurement and loading with average value for measurements and loading data.

In [4]:
measurements=[f'measurement_{i}' for i in range(0,0)]
measurements.append('loading')
for i in measurements:
    failure_data[i].fillna(failure_data[i].mean(),inplace=True)
    test_data[i].fillna(failure_data[i].mean(),inplace=True)

* Choosing the input fitting parameters for model and tranforming them by standard 
    scaler so that our model takes less time for training

In [5]:
test_col=measurements
X_train=failure_data[test_col]
Y_train=failure_data['failure']
X_test=test_data[test_col]

scaler = StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)

* Choosing the ML model for data
* U can just change index to see the performance for various models
* The model with best AUC score can be choosen as our model
* Tried neural network as well
* Varying its different parameters

In [6]:
neigh = KNeighborsClassifier(n_neighbors=3)
log= LogisticRegression()
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                   hidden_layer_sizes=(5,2), random_state=1)
clf.n_features_in_=5
models=[clf,log,neigh]
model= models[1]

* splitting the data into 2 parts
    * Train Data
    * Test Data

In [7]:
x, x_test, y, y_test = train_test_split(X_train,Y_train,test_size=0.2,random_state=17)

* Training the model by training data
* Testing the model for test (cross validation) data to check AUC score
* Maximizing the auc-score by varying the hyper-parameters




In [8]:
model.fit(x,y)
y_pred=model.predict(x_test)
print(y_pred)
score = roc_auc_score(y_pred, y_test)
print(f"ROC AUC: {score:.4f}")


[0. 0. 0. ... 0. 0. 0.]
ROC AUC: 0.6937


* Visualizing the predictions by Confusion Matrix

In [9]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[2533,    2],
       [ 684,    3]])

* Making the test_data ready for prediction by selecting , filling and transformation of specific colums of test data.
* Using the trained model to predict failure


In [10]:
X_test_train=test_data[measurements]
X_test_train=scaler.transform(X_test_train)
test_pred=model.predict_proba(X_test_train)
test_res=model.predict(X_test_train)
test_res.sum()


13.0

* Making the table of failures and the csv file for evaluation

In [11]:
series_id = pd.Series(test_data.id)
series_res=pd.Series(test_pred[:,1])
data=dict({
    "id":series_id,
    "failure":series_res
})
final_df = pd.concat(data,axis = 1)
final_df=final_df.set_index("id")
final_df.to_csv('results.csv')
final_df

,failure
id,
16108,0.231867
16109,0.233613
16110,0.196730
16111,0.198848
16112,0.202332
...,...
26565,0.251587
26566,0.233168
26567,0.193658
